In [ ]:
# EDA/데이터 전처리

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import requests as re

In [ ]:
df = pd.read_csv('vgames2.csv')

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df = df.drop(['Unnamed: 0'], axis=1)

In [ ]:
df['Name'].duplicated().sum()

In [ ]:
# 게임 이름 기준 중복 값 제거
df = df.drop_duplicates(['Name']).reset_index(drop=True)

In [ ]:
df.duplicated().sum()

In [ ]:
df.info()

In [ ]:
df

In [ ]:
# 제거하고자 하는 문자열 리스트 (검색 시 나오지 않거나 찾기 힘든 게임들 삭제)
example_list = ["Disney's Cinderella: Magical Dreams", "Chou Soujuu Mecha MG", "Swords", "The Smurfs", "Sonic the Hedgehog", "AKB1/48: Idol to Guam de Koishitara...", "The Hidden", "Hakuouki: Shinsengumi Kitan", "Indy 500", "Clockwork Empires", "Tornado", "Tour de France 2011", "Karate"]

In [ ]:
for i in example_list:
  id = df.loc[df.Name == i]
  df.drop(id.index, inplace=True)
  df.reset_index(drop=True, inplace=True)

In [ ]:
df.info()

In [ ]:
# 년도 결측치 인덱스 확인
yni = df[df['Year'].isnull()].index

In [ ]:
yni

In [ ]:
names = df['Name'].str.replace(' ', '_')

In [ ]:
# 출시 년도 웹크롤링
y = []

for i in yni:
  base = 'https://www.wikiwand.com/en/'
  url = names[i]
  headers = {'useragent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36'}        
  response = re.get(base+url, headers=headers)
  html = BeautifulSoup(response.text, "lxml")
  table = html.select("table")
  
  if table:    
    tables = pd.read_html(str(table))
    t = tables[0]
    a = t.loc[(t[t.columns[0]] =='First release') | (t[t.columns[0]] =='release') | (t[t.columns[0]] =='Released') | (t[t.columns[0]] =='Release') | (t[t.columns[0]] =='Publication date') | (t[t.columns[0]] =='Release dates') | (t[t.columns[0]] =='Release date')]
    if a.index == '':
      response = re.get(base+url+'(video game)', headers=headers)        
      html = BeautifulSoup(response.text, "lxml")        
      table = html.select("table")
      tables = pd.read_html(str(table))
      t = tables[0]

    a = t.loc[(t[t.columns[0]] =='First release') | (t[t.columns[0]] =='release') | (t[t.columns[0]] =='Released') | (t[t.columns[0]] =='Release') | (t[t.columns[0]] =='Publication date') | (t[t.columns[0]] =='Release dates') | (t[t.columns[0]] =='Release date')]
    s = a.loc[:, [t.columns[1]]]
    l = list(s.loc[s.index[0]])
    if l[0][-1] == ']':
      l = l[0][len(str(l[0]))-7:-3]
    else:
      l = l[0][-4:]
    y.append(l)
  else:
    y.append('')

In [ ]:
yni

In [ ]:
for i, v in enumerate(yni):
  df.loc[v, 'Year'] = y[i]+'.0'

In [ ]:
df

In [ ]:
df['Year'].isnull().sum()

In [ ]:
df.info()

In [ ]:
df2 = df.copy();

In [ ]:
df2['Year'] = df2['Year'].astype(str)
df2['Year']

In [ ]:
# Year값이 숫자가 아닌 행 삭제
wrong_RY = df2[df2['Year'].str.contains('[a-zA-Z]')].index

df2.drop(wrong_RY, inplace=True)
df2.reset_index(drop=True, inplace=True)
df2

In [ ]:
df2.Year.isnull().sum()

In [ ]:
# 이상치 제거
erase = ['1[1].0','][2].0','.0','006 .0']

for i in erase:
  e = df2.index[df2['Year']==i][0]
  df2.drop(e, inplace=True)
  df2.reset_index(drop=True, inplace=True)

In [ ]:
df2.info()

In [ ]:
df2['Year'] = df2['Year'].astype(float)
df2.info()

In [ ]:
df2.isnull().sum()

In [ ]:
df2

In [ ]:
# 년도 검색 시 나오지 않는 게임 제거
example_list2 = ["Disney's Aladdin","Mobile Suit Gundam"]

In [ ]:
for i in example_list2:
  id2 = df2.loc[df2.Name == i]
  df2.drop(id2.index, inplace=True)
  df2.reset_index(drop=True, inplace=True)

In [ ]:
df2.info()

In [ ]:
# 년도가 10의 자리로 되어있는 것들만 다시 크롤링
ind = []
for i in df2.Year.unique():
  if i // 1000 == 0:
    ii = df2[df2['Year']==i].index
    for j in range(len(ii)):
      ind.append(ii[j])

In [ ]:
# 크롤링을 위해 형태 변환
Names = df2['Name'].str.replace(' ', '_')

In [ ]:
# 출시 년도 웹크롤링2
y = []

for i in ind:
  base = 'https://www.wikiwand.com/en/'
  url = Names[i]
  headers = {'useragent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36'}        
  response = re.get(base+url, headers=headers)
  html = BeautifulSoup(response.text, "lxml")
  table = html.select("table")
  
  if table:    
    tables = pd.read_html(str(table))
    t = tables[0]
    a = t.loc[(t[t.columns[0]] =='First release') | (t[t.columns[0]] =='release') | (t[t.columns[0]] =='Released') | (t[t.columns[0]] =='Release') | (t[t.columns[0]] =='Publication date') | (t[t.columns[0]] =='Release dates') | (t[t.columns[0]] =='Release date')]
    print(url)
    if a.index == '':
      response = re.get(base+url+'(video game)', headers=headers)        
      html = BeautifulSoup(response.text, "lxml")        
      table = html.select("table")
      tables = pd.read_html(str(table))
      t = tables[0]
      print(t.columns[0])
    a = t.loc[(t[t.columns[0]] =='First release') | (t[t.columns[0]] =='release') | (t[t.columns[0]] =='Released') | (t[t.columns[0]] =='Release') | (t[t.columns[0]] =='Publication date') | (t[t.columns[0]] =='Release dates') | (t[t.columns[0]] =='Release date')]
    s = a.loc[:, [t.columns[1]]]
    l = list(s.loc[s.index[0]])
    if l[0][-1] == ']':
      l = l[0][len(str(l[0]))-7:-3]
    else:
      l = l[0][-4:]
    y.append(l)
    print(t.columns[0])
  else:
    y.append('')

In [ ]:
y2 = y

In [ ]:
y2

In [ ]:
for i, v in enumerate(ind):
  df2.loc[v, 'Year'] = y2[i]+'.0'

In [ ]:
# 이상치 제거
erase = ['tes).0','ease.0','.0']

for i in erase:
  e = df2.index[df2['Year']==i]
  df2.drop(e, inplace=True)
  df2.reset_index(drop=True, inplace=True)

In [ ]:
ee = df2.index[df2['Year']=='.0']
df2.drop(ee, inplace=True)
df2.reset_index(drop=True, inplace=True)

In [ ]:
df2.info()

In [ ]:
df2['Year'].value_counts()

In [ ]:
df2['Genre'].value_counts()

In [ ]:
# 장르 검색 시 나오지 않는 게임 제거
example_list3 = ["Stranglehold"]

In [ ]:
for i in example_list3:
  id2 = df2.loc[df2.Name == i]
  df2.drop(id2.index, inplace=True)
  df2.reset_index(drop=True, inplace=True)

In [ ]:
# 장르 비워있는 인덱스
gni = df2[df2['Genre'].isnull()].index
gni

In [ ]:
# 장르 웹 크롤링
g = []

for i in gni:
  base = 'https://www.wikiwand.com/en/'
  url = Names[i]
  headers = {'useragent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36'}        
  response = re.get(base+url, headers=headers)
  html = BeautifulSoup(response.text, "lxml")
  table = html.select("table")
  
  if table:    
    tables = pd.read_html(str(table))
    t = tables[0]
    a = t.loc[(t[t.columns[0]] =='Genre(s)') | (t[t.columns[0]] =='Genre')]
    if a.index == '':
      response = re.get(base+url+'(video game)', headers=headers)        
      html = BeautifulSoup(response.text, "lxml")        
      table = html.select("table")
      tables = pd.read_html(str(table))
      t = tables[0]

    print(t.columns[0])
    print(Names[i])
    a = t.loc[(t[t.columns[0]] =='Genre(s)') | (t[t.columns[0]] =='Genre')]
    s = a.loc[:, [t.columns[1]]]
    l = list(s.loc[s.index[0]])
    g.append(l)
  else:
    g.append('')

In [ ]:
g

In [ ]:
# 장르 채워넣기
for i, v in enumerate(gni):
  
  if g[i] != '':
    print(v)
    df2.loc[v, 'Genre'] = g[i][0]
    print(df2.loc[v, 'Genre'])

In [ ]:
df2

In [ ]:
# 퍼블리셔 삭제(의미 없다 판단)
df2 = df2.drop(['Publisher'], axis=1)

In [ ]:
df2.info()

In [ ]:
# Action이 장르명에 포함되면 Action으로 통일
r = df2[df2['Genre'].str.contains('Action', na=False)].index
for i in r:
  df2.loc[i, 'Genre'] = 'Action'

In [ ]:
# Sports가 장르명에 포함되면 Sports로 통일
s = df2[df2['Genre'].str.contains('Sports', na=False)].index
for i in s:
  df2.loc[i, 'Genre'] = 'Sports'

In [ ]:
# role-playing이 장르명에 포함되면 Role-Playing으로 통일
p = df2[df2['Genre'].str.contains('role-playing', na=False)].index
for i in p:
  df2.loc[i, 'Genre'] = 'Role-Playing'

In [ ]:
# shooter가 장르명에 포함되면 Shooting으로 통일
F = df2[df2['Genre'].str.contains('shooter|Shooter', na=False)].index
for i in F:
  df2.loc[i, 'Genre'] = 'Shooting'

In [ ]:
df2.isnull().sum()

In [ ]:
# blank 결측치 날리기
df2['Genre'].replace('', np.nan, inplace=True)
df2.dropna(subset=['Genre'], axis=0, inplace=True)

In [ ]:
df2.info()

In [ ]:
df2.reset_index(drop=True, inplace=True)

In [ ]:
df2

In [ ]:
df3 = df2.copy();

In [ ]:
df3.info()

In [ ]:
# 판매량 이상치 변환
sales = ['NA_Sales','EU_Sales','JP_Sales','Other_Sales']

In [ ]:
na = df3[df3['NA_Sales'].str.contains('K')].index
eu = df3[df3['EU_Sales'].str.contains('K')].index
jp = df3[df3['JP_Sales'].str.contains('K')].index
other = df3[df3['Other_Sales'].str.contains('K')].index

In [ ]:
# k = *1000 이므로 000, M은 ''로 변환 (M단위로 통일)
for i in sales:
  df3[i] = df3[i].str.replace('K', '000')
  df3[i] = df3[i].str.replace('M', '')
  df3[i] = df3[i].astype('float')


In [ ]:
df3

In [ ]:
# M 단위로 계산하기 위하여 K를 변환한 값들만 1000000으로 나눠준다.

In [ ]:
for i in na:
  df3.loc[i, 'NA_Sales'] = df3.loc[i, 'NA_Sales'] / 1000000
for i in eu:
  df3.loc[i, 'EU_Sales'] = df3.loc[i, 'EU_Sales'] / 1000000
for i in jp:
  df3.loc[i, 'JP_Sales'] = df3.loc[i, 'JP_Sales'] / 1000000
for i in other:
  df3.loc[i, 'Other_Sales'] = df3.loc[i, 'Other_Sales'] / 1000000

In [ ]:
df3

In [ ]:
# 결측치 및 이상치 모두 제거/변환 완료
df3.info()

In [ ]:
# 저장
df3.to_csv('final.csv')